读取all.csv文件

In [88]:
import os
import datetime
from rich.console import Console

console = Console()
DIR = os.path.abspath('')  # 在ipynb中，DIR的获取需要用abspath的方式，传参为空。在普通的python文件里，可以直接读取dirname。这个差异，可能跟ipynb文件的内部结构有关。
read_auto = open(DIR+'\\all.csv', 'r', encoding='utf8')

把all.csv文件的内容，逐行读取进total_data的列表。  
列表构建成3列，分别是时间（文本格式），物品名，和一个空列。  
空列的用途是对第1列进行时间格式的转换，转换成时间戳形式，目的是用于排序和查重

In [89]:
total_data = []
for line in read_auto:
    line = line.strip()
    if line != '':
        try:
            # 用于处理某些数据异常情况，例如2条记录在同一行
            if ',' in line:
                time_, item_ = line.split(',')
                total_data.append([time_, item_, 0])
        except Exception as e:
            console.print(line)

read_auto.close()

对 total_data 里的第1列进行转化，转成时间戳。
转化后，存入到 total_data 的第3列
然后按照第3列进行排序

datetime库的用法 和 time库的用法相似。

In [90]:
for each in total_data:
    # 把文本时间值 格式化成 时间值类型
    time_ = datetime.datetime.strptime(each[0], '%Y-%m-%d %H:%M:%S')
    # 时间值类型 转换成 时间戳
    time_stamp = datetime.datetime.timestamp(time_)
    each[2] = int(time_stamp)

total_data.sort(key=lambda x:x[2])

total_data[-5:]

[['2022-12-19 15:31:12', '贝斯', 1671435072],
 ['2022-12-19 15:32:10', '钢琴', 1671435130],
 ['2022-12-19 15:33:08', '小提琴', 1671435188],
 ['2022-12-19 15:34:06', '圆号', 1671435246],
 ['2022-12-19 15:35:04', '小提琴', 1671435304]]

建立2个列表。  
unique用于保存不重复的值，remove用于保存重复的值。  
根据时间值判断是否发生了重复。

In [91]:
unique = []
remove = []
for each in total_data:
    if each[2] in unique:
        remove.append(each)
    else:
        unique.append(each[2])

# 把重复的值从total_data中移除
for each in remove:
    total_data.remove(each)

out = open(DIR+'\\deal_result.csv', 'w', encoding='utf8')

for i in range(len(total_data)):
    # if i > 0:
    #     if total_data[i][2] - total_data[i-1][2] > 58:
    #         out.write('\n')
    out.write(total_data[i][0]+','+total_data[i][1]+'\n')

out.close()

定义一个按时间间隔大小分割数据的函数。  
如果两条相邻记录的时间间隔超过58秒，则认为这两条记录分别属于2个不同的数据集部分。  
按照这个方式把total_data分割成不同的部分，存为total_data_split

In [86]:
total_data_split = []
_middle = []
for i in range(len(total_data)-1):
    _middle.append(total_data[i])
    if total_data[i+1][2] - total_data[i][2] > 58:
        total_data_split.append(_middle)
        _middle = []

total_data_split.append(_middle)
_middle = []

对 total_data_split 进行进一步的加工。  
取其每一部分，先判断总量是否超过20。如果没超过，则数据太短，舍弃。  
如果超过了，则在这个部分内，按每次取21个值，重构成一个列表数据，前20项表示当前的历史记录（20条），第21项表示对应的结果。

In [87]:
_middle = []
history = []

for each in total_data_split:
    if len(each) > 20:
        _middle.append(each)
    else:
        pass

for each in _middle:
    for i in range(len(each)-21+1):
        history.append(each[i:22+i])

history[-1]

[['2022-12-19 15:14:46', '吉他', 1671434086],
 ['2022-12-19 15:15:44', '钢琴', 1671434144],
 ['2022-12-19 15:16:42', '吉他', 1671434202],
 ['2022-12-19 15:17:40', '吉他', 1671434260],
 ['2022-12-19 15:18:38', '小提琴', 1671434318],
 ['2022-12-19 15:19:36', '小提琴', 1671434376],
 ['2022-12-19 15:20:34', '贝斯', 1671434434],
 ['2022-12-19 15:21:32', '钢琴', 1671434492],
 ['2022-12-19 15:22:30', '架子鼓', 1671434550],
 ['2022-12-19 15:23:28', '贝斯', 1671434608],
 ['2022-12-19 15:24:26', '贝斯', 1671434666],
 ['2022-12-19 15:25:24', '贝斯', 1671434724],
 ['2022-12-19 15:26:22', '竖琴', 1671434782],
 ['2022-12-19 15:27:20', '钢琴', 1671434840],
 ['2022-12-19 15:28:18', '贝斯', 1671434898],
 ['2022-12-19 15:29:16', '钢琴', 1671434956],
 ['2022-12-19 15:30:14', '钢琴', 1671435014],
 ['2022-12-19 15:31:12', '贝斯', 1671435072],
 ['2022-12-19 15:32:10', '钢琴', 1671435130],
 ['2022-12-19 15:33:08', '小提琴', 1671435188],
 ['2022-12-19 15:34:06', '圆号', 1671435246]]